In [1]:
import tensorflow as tf
print(tf.__version__)

2.3.1


We will again be using the sklearn diabetes dataset to demonstrate these callbacks.

In [2]:
# Load the diabetes dataset

from sklearn.datasets import load_diabetes

diabetes_dataset = load_diabetes()

In [3]:
# Save the input and target variables

from sklearn.model_selection import train_test_split

data = diabetes_dataset['data']
targets = diabetes_dataset['target']

In [4]:
# Split the data set into training and test sets

train_data, test_data, train_targets, test_targets = train_test_split(data, targets, test_size=0.1)

Let's also build a simple model to fit to the data with our callbacks.

In [5]:
# Build the model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = tf.keras.Sequential([
    Dense(128, activation='relu', input_shape=(train_data.shape[1],)),
    Dense(64,activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)        
])

In [6]:
# Compile the model

model.compile(loss='mse',
                optimizer="adam",metrics=["mse","mae"])

Now onto the callbacks!

In [7]:
def lr_function(epoch, lr):
    if epoch%2==0:
        return lr
    else:
        return lr + epoch/1000

In [9]:
history = model.fit(train_data, train_targets, epochs=10,
                   callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_function, verbose=1)], verbose=False)


Epoch 00001: LearningRateScheduler reducing learning rate to 0.0010000000474974513.

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0020000000474974513.

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0020000000949949026.

Epoch 00004: LearningRateScheduler reducing learning rate to 0.005000000094994903.

Epoch 00005: LearningRateScheduler reducing learning rate to 0.004999999888241291.

Epoch 00006: LearningRateScheduler reducing learning rate to 0.009999999888241292.

Epoch 00007: LearningRateScheduler reducing learning rate to 0.009999999776482582.

Epoch 00008: LearningRateScheduler reducing learning rate to 0.01699999977648258.

Epoch 00009: LearningRateScheduler reducing learning rate to 0.016999999061226845.

Epoch 00010: LearningRateScheduler reducing learning rate to 0.025999999061226846.


In [10]:
history = model.fit(train_data, train_targets, epochs=10,
                    callbacks=[tf.keras.callbacks.LearningRateScheduler(lambda x:1/(3+5*x), verbose=1)], 
                    verbose=False)


Epoch 00001: LearningRateScheduler reducing learning rate to 0.3333333333333333.

Epoch 00002: LearningRateScheduler reducing learning rate to 0.125.

Epoch 00003: LearningRateScheduler reducing learning rate to 0.07692307692307693.

Epoch 00004: LearningRateScheduler reducing learning rate to 0.05555555555555555.

Epoch 00005: LearningRateScheduler reducing learning rate to 0.043478260869565216.

Epoch 00006: LearningRateScheduler reducing learning rate to 0.03571428571428571.

Epoch 00007: LearningRateScheduler reducing learning rate to 0.030303030303030304.

Epoch 00008: LearningRateScheduler reducing learning rate to 0.02631578947368421.

Epoch 00009: LearningRateScheduler reducing learning rate to 0.023255813953488372.

Epoch 00010: LearningRateScheduler reducing learning rate to 0.020833333333333332.


In [11]:
history = model.fit(train_data, train_targets, epochs=10, 
                   callbacks=[tf.keras.callbacks.CSVLogger('res.csv')], verbose=False)

In [12]:
import pandas as pd
pd.read_csv('res.csv', index_col='epoch')

,loss,mae,mse
epoch,,,
0,2981.337158,44.488266,2981.337158
1,2980.894287,44.546947,2980.894287
2,2958.102295,44.112652,2958.102295
3,2993.965088,44.505032,2993.965088
4,2951.246582,44.274509,2951.246582
5,2964.477539,44.255322,2964.477539
6,2956.551758,44.226921,2956.551758
7,2992.910645,44.481030,2992.910645
8,2974.920410,44.286129,2974.920410


In [13]:
epoch_callback = tf.keras.callbacks.LambdaCallback(on_epoch_begin=lambda epoch, logs: 
                                                   print(f'Starting Epoch {epoch+1}'))

In [17]:
batch_loss_callback = tf.keras.callbacks.LambdaCallback(on_batch_end=lambda batch, logs:
                                                       print(f'After batch {batch}, the loss is {logs["loss"]:.4f}'))

In [18]:
train_finish_callback = tf.keras.callbacks.LambdaCallback(
    on_train_end=lambda logs: print('Training finished!'))

In [19]:
# Train the model with the lambda callbacks

history = model.fit(train_data, train_targets, epochs=5, batch_size=100,
                    callbacks=[epoch_callback, batch_loss_callback,train_finish_callback], verbose=False)

Starting Epoch 1
After batch 0, the loss is 3028.8181
After batch 1, the loss is 3096.2200
After batch 2, the loss is 2994.4788
After batch 3, the loss is 2959.3142
Starting Epoch 2
After batch 0, the loss is 2641.7471
After batch 1, the loss is 2779.2043
After batch 2, the loss is 2954.2251
After batch 3, the loss is 2943.9089
Starting Epoch 3
After batch 0, the loss is 2693.1860
After batch 1, the loss is 3099.1619
After batch 2, the loss is 3045.7583
After batch 3, the loss is 2933.3735
Starting Epoch 4
After batch 0, the loss is 3334.1638
After batch 1, the loss is 3054.3115
After batch 2, the loss is 2858.5930
After batch 3, the loss is 2948.0161
Starting Epoch 5
After batch 0, the loss is 3138.0693
After batch 1, the loss is 2929.6570
After batch 2, the loss is 3016.6877
After batch 3, the loss is 2940.6101
Training finished!


In [21]:
history = model.fit(train_data, train_targets, epochs=100, batch_size=100,
                   callbacks=[tf.keras.callbacks.ReduceLROnPlateau(
                   monitor='loss', factor=0.2, verbose=1)], verbose=False)


Epoch 00026: ReduceLROnPlateau reducing learning rate to 0.00416666679084301.

Epoch 00036: ReduceLROnPlateau reducing learning rate to 0.0008333333767950535.

Epoch 00056: ReduceLROnPlateau reducing learning rate to 0.00016666667070239783.

Epoch 00092: ReduceLROnPlateau reducing learning rate to 3.333333297632635e-05.
